# Sonar - Decentralized Model Training Simulation (local)

DISCLAIMER: This is a proof-of-concept implementation. It does not represent a remotely product ready implementation or follow proper conventions for security, convenience, or scalability. It is part of a broader proof-of-concept demonstrating the vision of the OpenMined project, its major moving parts, and how they might work together.


# Getting Started: Installation

##### Step 1: install IPFS

- https://ipfs.io/docs/install/

##### Step 2: Turn on IPFS Daemon
Execute on command line:
> ipfs daemon

##### Step 3: Install Ethereum testrpc

- https://github.com/ethereumjs/testrpc

##### Step 4: Turn on testrpc with 1000 initialized accounts (each with some money)
Execute on command line:
> testrpc -a 1000

##### Step 5: install openmined/sonar and all dependencies
##### Step 6: execute the following code

# The Simulation: Diabetes Prediction

In this example, a diabetes research center (Cure Diabetes Inc) wants to train a model to try to predict the progression of diabetes based on several indicators. They have collected a small sample (42 patients) of data but it's not enough to train a model. So, they intend to offer up a bounty of $5,000 to the OpenMined commmunity to train a high quality model.

As it turns out, there are 400 diabetics in the network who are candidates for the model (are collecting the relevant fields). In this simulation, we're going to faciliate the training of Cure Diabetes Inc incentivizing these 400 anonymous contributors to train the model using the Ethereum blockchain.

Note, in this simulation we're only going to use the sonar and syft packages (and everything is going to be deployed locally on a test blockchain). Future simulations will incorporate mine and capsule for greater anonymity and automation.

### Imports and Convenience Functions

In [1]:
import warnings
import numpy as np
import phe as paillier
from sonar.contracts import ModelRepository,Model
from syft.he.Paillier import KeyPair
from syft.nn.linear import LinearClassifier
import numpy as np
from sklearn.datasets import load_diabetes

def get_balance(account):
    return repo.web3.fromWei(repo.web3.eth.getBalance(account),'ether')

### Setting up the Experiment

In [2]:
# for the purpose of the simulation, we're going to split our dataset up amongst
# the relevant simulated users

diabetes = load_diabetes()
y = diabetes.target
X = diabetes.data

validation = (X[0:42],y[0:42])
anonymous_diabetes_users = (X[42:],y[42:])

# we're also going to initialize the model trainer smart contract, which in the
# real world would already be on the blockchain (managing other contracts) before
# the simulation begins

repo = ModelRepository() # blockchain hosted model repository

# we're going to set aside 400 accounts for our 400 patients
# Let's go ahead and pair each data point with each patient's 
# address so that we know we don't get them confused
patient_addresses = repo.web3.eth.accounts[100:500]
anonymous_diabetics = list(zip(patient_addresses,
                               anonymous_diabetes_users[0],
                               anonymous_diabetes_users[1]))

# we're going to set aside 1 account for Cure Diabetes Inc
cure_diabetes_inc = repo.web3.eth.accounts[501]

No account submitted... using default[2]
Deployed ModelRepository:0x79c8d937ac7ce82166e3d0c225b9313e1910d18b17cc632cad13d4f46103d732


### Step 1: Cure Diabetes Inc Initializes a Model and Provides a Bounty

In [3]:
warnings.filterwarnings('ignore')

In [4]:
bounty = 0.001 # 5 metacoins

In [5]:
# current balance of Cure Diabetes Inc
get_balance(cure_diabetes_inc)

Decimal('89.985999999997328513')

In [6]:
pubkey,prikey = KeyPair().generate(n_length=1024)
diabetes_classifier = LinearClassifier(desc="DiabetesClassifier",n_inputs=10,n_labels=1)
initial_error = diabetes_classifier.evaluate(validation[0],validation[1])
diabetes_classifier.encrypt(pubkey)

diabetes_model = Model(owner=cure_diabetes_inc,
                       syft_obj = diabetes_classifier,
                       bounty = bounty,
                       initial_error = initial_error,
                       target_error = 10000
                      )

In [7]:
model_id = repo.submit_model(diabetes_model)

In [8]:
# notice that the current balance of Cure Diabetes Inc goes down by the size of the bounty + gas
get_balance(cure_diabetes_inc)

Decimal('89.984999999997160961')

## Step 2: An Anonymous Patient Downloads the Model and Improves It

In [9]:
patient = anonymous_diabetics[0]

addr,input,target = patient
get_balance(addr)

encrypted_diabetes_model = repo[model_id]

In [10]:
grad = encrypted_diabetes_model.generate_gradient(input,target)
grad_id = repo.submit_gradient(addr,model_id,grad)

## Step 3: Cure Diabetes Inc. Evaluates the Gradient 

In [11]:
diabetes_model = repo[model_id]

In [19]:
g = diabetes_model[0]

In [21]:
g

In [22]:
repo.call.getGradient(0,0)

['0x076ecdd7a2e917644b450da2df38d9342c28fe70',
 ['QmYRizTGmrcV2YENgFknzvAjvSDrw4vx',
  'CEjRu4odK8sPb4\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00']]